# Import Packages

In [1]:
import pandas as pd
import numpy as np
import turicreate as tc

# Read the Files and Preparing the Data

In [2]:
post = pd.read_csv('posts (1).csv')
user_ratings = pd.read_csv('userRatings.csv', usecols=['user_id', 'post_id', 'Ratings'])

Display the first 5 rows of the post dataset

In [3]:
post.head(5)

,user_id,title,content,tags,slug,created_at,updated_at,image,status_id,action,post_id
0,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,NaN,what-i-have-learnt-so-far-on-html-985,2019-08-25 19:39:01,2019-08-28 11:30:00,1,NaN,NaN,NaN
1,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ...",Technology,html-begins-here-4d6,2019-08-25 19:44:36,2019-08-28 11:30:00,1,NaN,NaN,NaN
2,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,NaN,my-laziness-in-the-open-029,2019-08-25 19:50:33,2019-08-28 11:30:00,1,NaN,NaN,NaN
3,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,NaN,my-task-2-649,2019-08-25 19:51:36,2019-08-28 11:30:00,1,NaN,NaN,NaN
4,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str...",NaN,task-2-7e5,2019-08-25 19:51:39,2019-08-28 11:30:00,1,NaN,NaN,NaN


Drop this columns because they are not relevant to our model

In [4]:
col = ['tags', 'slug', 'created_at', 'updated_at', 'image', 'status_id', 'action', 'post_id']

postdf = post.drop(post[col], axis=1)

Read the first 5 cols of the new dataset

In [5]:
postdf.head()

,user_id,title,content
0,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...
1,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ..."
2,1310,My Laziness In The Open,I have not been attending classes on the HNG c...
3,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...
4,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str..."


Display the first 5 rows of the user_ratings dataset

In [6]:
user_ratings.head()

,user_id,post_id,Ratings
0,1,995.0,5
1,1,355.0,2
2,1,996.0,4
3,1,996.0,5
4,1,996.0,4


Merge our postdf and user ratings df

In [7]:
merged_col = pd.merge(postdf, user_ratings, on='user_id')

Display the head of our merged dataframe

In [8]:
merged_col.head()

,user_id,title,content,post_id,Ratings
0,2191,What you have learned so far,"1. I have learnt how to publish and article, ...",143.0,3
1,2191,What you wish to still learn,1. I want more insight on clodinary and so on,143.0,3
2,2191,What you wish should be improved,1. For now am OK with the setting,143.0,3
3,4,Good Afternoon People,Hey,999.0,4
4,4,Good Afternoon People,Hey,278.0,3


In [9]:
#number of users

users = post['user_id'].unique() ; print("number of users" , len(users))

number of users 308


In [10]:
#number of content titles

titles = post['title'].unique() ; print("Number of titles" , len(titles))

Number of titles 658


In [11]:
#train_data, test_data = train_test_split(merged_col, test_size = 0.20, random_state=0)

Convert merged data to csv

In [12]:
merged_col.to_csv('merged_data.csv')

Reading back the csv file and converting it to Sframe for use by the turicreate model

In [13]:
data = tc.SFrame.read_csv('merged_data.csv')

Finished parsing file /home/godwin/Downloads/luci/merged_data.csv

Parsing completed. Parsed 100 lines in 0.048791 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,str,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/godwin/Downloads/luci/merged_data.csv

Parsing completed. Parsed 622 lines in 0.03625 secs.

# Model 

Creating a train and validation data for our model

In [14]:
trainind_data, validation_data = tc.recommender.util.random_split_by_user(data, 'user_id', 'title')

Building the model using the ranking factorization recommender

In [15]:
rating_model = tc.ranking_factorization_recommender.create(trainind_data, 
                                                    user_id ='user_id',
                                                   item_id ='title',
                                                   target = 'Ratings')

Preparing data set.

Data has 505 observations with 8 users and 45 items.

Data prepared in: 0.012477s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.51953

Final training RMSE: 0.730375

Creating an item_similarity model, So we can compare the two models together

In [16]:
item_sim_model = tc.item_similarity_recommender.create(trainind_data, user_id='user_id', 
                                                       item_id='content', 
                                                       target='Ratings', similarity_type='cosine')

Warning: Ignoring columns X1, title, post_id;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 505 observations with 8 users and 51 items.

Data prepared in: 0.008994s

Training model from provided data.

Gathering per-item and per-user statistics.

# Compare Both Models

In [17]:
tc.recommender.util.compare_models(validation_data, [rating_model,item_sim_model], metric='rmse')

+--------------------------------+------------+

| Elapsed Time (Item Statistics) | % Complete |

+--------------------------------+------------+

| 357us                          | 100        |

+--------------------------------+------------+

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.002035s

PROGRESS: Evaluate model M0

Overall RMSE: 0.7243378040721419

Per User RMSE (best)
+---------+--------------------+-------+
| user_id |        rmse        | count |
+---------+--------------------+-------+
|    3    | 0.6423130812472058 |   43  |
+---------+--------------------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+---------+--------------------+-------+
| user_id |        rmse        | count |
+---------+--------------------+-------+
|    7    | 0.8571471484665503 |   7   |
+---------+--------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (best)
+-------------------+---------------------+-------+
|       title       |         rmse        | count |
+-------------------+---------------------+-------+
| Took a long while | 0.07714612298590673 |   1   |
+-------------------+---------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (worst)
+--------+--------------------+-------+
| title  |        rmse        | count |
+--------+--------------------+-

[{'rmse_by_user': Columns:
  	user_id	int
  	rmse	float
  	count	int
  
  Rows: 4
  
  Data:
  +---------+--------------------+-------+
  | user_id |        rmse        | count |
  +---------+--------------------+-------+
  |    2    | 0.7565610281285797 |   65  |
  |    3    | 0.6423130812472058 |   43  |
  |    1    | 0.8054768050054123 |   2   |
  |    7    | 0.8571471484665503 |   7   |
  +---------+--------------------+-------+
  [4 rows x 3 columns], 'rmse_by_item': Columns:
  	title	str
  	rmse	float
  	count	int
  
  Rows: 29
  
  Data:
  +-------------------------------+---------------------+-------+
  |             title             |         rmse        | count |
  +-------------------------------+---------------------+-------+
  |         Lucid is back         | 0.23842351122971533 |   3   |
  |         Stressful Day         |  0.5139921567441065 |   2   |
  |      Hello New Followers      | 0.22008927529543548 |   2   |
  |      How did your day go?     |  0.28734826143005

We compared both models and we can see that the rating model performed better with a rmse of 0.71

# Recommender System Testing

*Making predictions based on different factors

1. Making recommendation for all users
    * This makes recommendations for all users seen during model creation
    * It returns the first 10 prediction

In [18]:
all_users_rank = rating_model.recommend()
all_users_rank

user_id,title,score,rank
2191,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.8108921568877636,1
2191,third,3.7278916552699743,2
2191,First,3.725977162461692,3
2191,Took a long while,3.7134096948234006,4
2191,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.6940262835509716,5
2191,second,3.670842039961332,6
2191,html test,3.649485618904197,7
2191,Welcome to Lucid,3.6079779207743345,8
2191,DON'T ASK WHY YOU WEREREJECTED ...,3.6024733284525214,9
2191,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.6003631079606353,10


* k=3 arguement means that it should return 3 predictions for each users
* It can basically be any amount of recommendations we want the recommender system to produce

In [19]:
all_users_rank = rating_model.recommend(k=3)
all_users_rank

user_id,title,score,rank
2191,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.8108921568877636,1
2191,third,3.7278916552699743,2
2191,First,3.725977162461692,3
4,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.736782988164538,1
4,third,3.670906284455174,2
4,First,3.6675673747472106,3
1,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.9372714977748333,1
1,html test,3.794475407436454,2
1,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.775977082181686,3
7,html test,3.634259300403261,1


2.Making recommendations for new users without any information
    * created a new user with user_id = 20000
    * asked the rec_sys to recommend 3 articles for the user

In [20]:
rank_factor = rating_model.recommend(users=[20000], k=3)
rank_factor

user_id,title,score,rank
20000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.77461164407801,1
20000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.705891161788219,2
20000,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.674616511542314,3


* it can also return a list of users

In [21]:
rank_factor = rating_model.recommend(users=[20000, 21000, 22000], k=3)
rank_factor

user_id,title,score,rank
20000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.77461164407801,1
20000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.705891161788219,2
20000,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.674616511542314,3
21000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.77461164407801,1
21000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.705891161788219,2
21000,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.674616511542314,3
22000,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,3.77461164407801,1
22000,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.705891161788219,2
22000,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.674616511542314,3
